# __Lecture 09: Practical analyses in Python (continued)__

----
#### __Announcement:__ Homework 4 will be finalized by the end of the day today (I will post on Slack when it's ready). It will be due on Nov 2

#### On Tuesday, we talked about:
- how to define functions in python
- the `re` module

#### Today we will:
- practice using the `re` module
- learn about (and practice) using the `biopython` module

###  
----

## __Review:__ Regular expressions

The `re` module offers a set of functions that allows us to search a string for a match. 
Here is the general format for searching for a pattern in a string:

```
import re
string = "This is an example string"

# compile the search pattern
search_pattern = re.compile("search pattern here")

# search for the search pattern in the string
search_pattern.search(string)
```

Here are some common elements to have in your search pattern:
* letter characters which returns a match where the string contains the specified letter (e.g. `A`, `B`, `C`, ...)
* special characters which returns a match where the string contains the specified special character; these must be preceded by a `\`
* `\d` which returns a match where the string contains digits (numbers from `0`-`9`)

You may also want to add the following customizations:
* `[]` specifies a set of characters to search for (e.g. `[a-n]`)
* `()` capture and group everything contained inside, and search the string for everything together
* `?P<name>` indicates a search pattern group with name `name`
* `+` specifies one or more occurrences of a certain pattern element
* `{}` specifies exactly the specified number of occurrences of a certain pattern element
* `$` specifies the end of the string
* `^` specifies the beginning of the string

## __Practice time:__ using regular expressions to parse barcodes
Now we will use regular expressions to parse barcodes from nucleotide sequences.
For instance, you might have to do this in a single-cell RNA-seq experiment where there is a barcode at the end of each read telling you the cell that the read came from.

Imagine that our valid molecules should have sequences like this:

`CTAGCNNNNNNGATCA`

See how there is a 6-nucleotide barcode in the center of the sequence.
We have a list of sequences, and want to parse through them to figure out which ones meet the expected pattern--and get the barcode from those that do:

In [3]:
seqs = ['CTAGCatcgatGATCA',  # has barcode ATCGAT
        'CCAGCatagcaGATCA',  # does not have expected 5' sequence
        'CTAGCtacagGATCA',   # barcode too short
        'CTAGCgaccatGATCA',  # has barcode GACCAT
        'CTAGCatcgatGATCA',  # has barcode ATCGAT
        'CTAGCatcgatGGTCA',  # does not have expected 3' sequence
        ]

Write a function that parses these barcoded sequences and gets the ones with valid barcodes.
In doing this, note that:

  1. If you have a string `s`, `s.upper()` makes it all uppercase.
  2. You may want to start searching at the beginning of the string (using the `^` symbol in the search pattern)
    
Below I've written the function documentation, try to implement it.
__Take a few minutes in groups to work through this.__

In [1]:
def count_barcodes(seqs, bclen=6, upstream='CTAGC', downstream='GATCA'):
    """Parse and count barcodes.
    
    Parameters
    ----------
    seqs : list
        DNA sequences.
    bclen : int
        Length of barcode
    upstream : str
        Sequence upstream of barcode.
    downstream : str
        Sequence downstream of barcode.
        
    Returns
    -------
    dict
        Keyed by each valid barcode, value is number of times the barcode
        is observed.
        
    Note
    ----
    The function is **not** case-sensitive, and all barcodes are reported
    in upper-case.
    
    """
    
        
    return None


Run the function once you've implemented it. Does it give the right result?

In [4]:
count_barcodes(seqs)

## Biopython
[Biopython](https://biopython.org/) is a package that has lots of useful functions for computational biology.

It is very handy for things like reading in sequences in many different formats: the subpackage [Bio.SeqIO](https://biopython.org/wiki/SeqIO) is your friend!

(Do note that if you are analyzing truly large datasets, `Biopython` is not very fast and you may want to use something like [pysam](https://pysam.readthedocs.io/en/latest/api.html); but `Biopython` is a good starting point).

### Reading in a file
I have included the file [barcodes_R1.fastq](barcodes_R1.fastq), which has some FASTQ sequences in it.

Let's use `Biopython` to read the FASTQ entries.

First, we'll need to import `Biopython.SeqIO`

In [50]:
import re

import Bio.SeqIO

Now read in the sequencing reads:

In [6]:
seqreads = list(Bio.SeqIO.parse('barcodes_R1.fastq', format='fastq'))

How many reads were there?

In [7]:
print(f"Found {len(seqreads)} sequencing reads.")

Found 10000 sequencing reads.


Let's look at the first read:

In [8]:
seqreads[0]

SeqRecord(seq=Seq('GCTTAAGTTATTTAGTGCGGCCGCCTATGGTGCACTATTATTTATCTATCGTGA...AGA'), id='HISEQ:621:HMJGNBCX2:1:1101:1797:2150', name='HISEQ:621:HMJGNBCX2:1:1101:1797:2150', description='HISEQ:621:HMJGNBCX2:1:1101:1797:2150 1:N:0:ATGTCA', dbxrefs=[])

You can see that `BioPython` reads the sequences in as [SeqRecord](https://biopython.org/wiki/SeqRecord) objects, which have a lot of information, including the header, quality scores, etc.

If you want to just access the sequence element of each `SeqRecord`, you can do this as follows:

In [32]:
seqreads[0].seq

Seq('GCTTAAGTTATTTAGTGCGGCCGCCTATGGTGCACTATTATTTATCTATCGTGA...AGA')

In [33]:
seqreads_Seq = []
for seqrecord in seqreads:
    sequence = seqrecord.seq # isolate the sequence from the seqrecord
    seqreads_Seq.append(sequence) # add string sequence to list

In [35]:
seqreads_Seq[0:5]

[Seq('GCTTAAGTTATTTAGTGCGGCCGCCTATGGTGCACTATTATTTATCTATCGTGA...AGA'),
 Seq('CTTCCTGGTCACGGTTGCGGCCGCCTATGGTGCATCATTATATGCAAATCCGGC...CGA'),
 Seq('CACGGCTATTACCCCGCGGCCGCCTATGGTGCACTATTATTTATCTATCGTGAA...CGA'),
 Seq('TCAGCGATGAATGTGGGCGGCCGCCTATGTTGCATCATTATATGCAAATCCGGC...TGA'),
 Seq('ATATGGGAGACGATAAGCGGCCGCCTATGGTGCATCATTATATGCAAATCCGGC...CTC')]

These sequences are a `Seq` objects. If you want to convert the sequences to strings, you can do so as follows:

In [39]:
str(seqreads_Seq[0])

'GCTTAAGTTATTTAGTGCGGCCGCCTATGGTGCACTATTATTTATCTATCGTGAAAGGGAGTTCTGCTCCATCAGGCCAAGATCGGAAGAGCACACGTCTGAACTCCAGTCACATGTCAGAATCTCGTATGCCGTCTTCTGCTTGAAAAAAAAAAAAAAGCTGAAATTAATAATTTTGAAACCAGTTTTGTAAACGCAGCACTAAAATGAAGGCATGCAACGACGATGTTTATTGACACGGAATAGCAGA'

Likewise, strings are not `Seq` objects. If you would like to create a `Seq` object from a string, you can do so as follows:

In [49]:
# This requires us to first import the Seq method
from Bio.Seq import Seq

Seq('ATTGCT')

Seq('ATTGCT')

### Built-in methods

`Biopython` has many useful built-in functions for working with sequencing data. 
We will discuss a few examples in class from the submodule [Bio.Seq](https://biopython.org/docs/1.75/api/Bio.Seq.html), but feel free to read about more [here](https://biopython.org/wiki/Seq)

In [42]:
seq = seqreads_Seq[0]
seq

Seq('GCTTAAGTTATTTAGTGCGGCCGCCTATGGTGCACTATTATTTATCTATCGTGA...AGA')

We can use this module to get the __complement__ and __reverse complement__ of a sequence:

In [41]:
seq.complement()

Seq('CGAATTCAATAAATCACGCCGGCGGATACCACGTGATAATAAATAGATAGCACT...TCT')

In [43]:
seq.reverse_complement()

Seq('TCTGCTATTCCGTGTCAATAAACATCGTCGTTGCATGCCTTCATTTTAGTGCTG...AGC')

We can use this module to __transcribe__ and __translate__ a sequence:

In [44]:
seq.transcribe()

Seq('GCUUAAGUUAUUUAGUGCGGCCGCCUAUGGUGCACUAUUAUUUAUCUAUCGUGA...AGA')

In [45]:
seq.translate()

/opt/conda/lib/python3.9/site-packages/Bio/Seq.py:2979: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn(


Seq('A*VI*CGRLWCTIIYLS*KGVLLHQAKIGRAHV*TPVTCQNLVCRLLLEKKKKA...E*Q')

__If you choose to use these methods, remember that a `Seq` object is not a string. You will need to convert your sequence back to a string before using methods/functions that require strings.__

## A real biological analysis: parsing barcodes
<a id='real_analysis'></a>
The reads that we just read as `seqreads_Seq` come from a real sequencing run of influenza virus HA and NA genes.

The __actual sequences__ are as follows:

    5'-[end of HA]-AGGCGGCCGC-[16 X N barcode]-3'
    
or 

    5'-[end of NA]-AGGCGGCCGC-[16 X N barcode]-3'
    
__The end of NA is:__

    ...CACGATAGATAAATAATAGTGCACCAT
    
__The end of HA is:__

    ...CCGGATTTGCATATAATGATGCACCAT

    
The __sequencing run reads__ from the reverse end of the molecules (in 5'>3' orientation), so the sequencing reads are as follows:

    5'-[reverse complement of 16 X N barcode]-GCGGCCGCCT-[reverse complement of the end of HA]-3'

or 

    5'-[reverse complement of 16 X N barcode]-GCGGCCGCCT-[reverse complement of the end of NA]-3'



We want to determine which reads have valid sequences, get the barcodes out of strings, and count the barcodes.
So this requires setting up an analysis that does the following:

 1. Get the reverse complement of each read.
 2. Determine if it matches the expected pattern (with the correct length barcode and constant sequence)
 3. If it matches, extract the barcode and add it to a dictionary to keep track of counts.

### Group activity
In groups, work together to write some code to do this.
I have created a code chunk for each step (with some parts filled in). 
Remember to run the code chunks in the correct order!

For your homework, you will be asked to extend this in-class analysis to get statistics for HA and NA seperately.

In [ ]:
# load necessary packages
import re
import Bio.SeqIO

__Step 1:__ You'll need to write a function that identifies a barcode with a known upstream sequence. 
I've provided the documentation here--try writing this function on your own.

_Hint: we wrote a similar function yesterday_

_Hint 2: You can use the built-in reverse complement method_

_Hint 3: You will want to convert the sequence to a string before searching for regular expressions_

In [51]:
def read_barcode(seqread, bclen, upstream='AGGCGGCCGC'):
    """Identify barcode with known upstream sequence.
    
    Parameters
    ----------
    seqread : Seq object
        Nucleotide sequence matching UPSTREAM-BARCODE read in reverse orientation.
    bclen : int
        Length of barcode
    upstream: str
        Sequence upstream of the barcode.
        
    Returns
    -------
    str or None
        Sequence of the barcode in the forward orientation, or `None` if no match to expected barcoded sequence.
        
    Example
    -------
    >>> read_barcode('TTTTTTTTTTTTTTTTGCGGCCGCCT', bclen=16)
    'AAAAAAAAAAAAAAAA'
        
    """

    return None

__Step 2:__ Read sequences from the barcodes_R1.fastq file and create a list of only the sequences (as Seq objects). _We already did this step earlier_

In [52]:
# run this code chunk...
seqreads = list(Bio.SeqIO.parse('barcodes_R1.fastq', 'fastq'))

seqreads_Seq = []
for seqrecord in seqreads:
    seqreads_Seq.append(seqrecord.seq)

__Step 3:__ Get the counts of all barcodes. _(Hint: you might want to store barcodes and counts in a dictionary, and also keep track of the number of sequences that don't have a valid barcode)_

Please name your dictionary `barcode_counts`

In [53]:
# your code here...

__Step 4:__ Report the total number of sequences parsed, and how many lacked a valid barcode.

In [54]:
# your code here...